### Stance classification was run using Colab

#### Install libraries

In [ ]:
!pip install torch torchvision
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel
import numpy as np
import torch
from scipy.special import softmax
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# mount the google drive 
from google.colab import drive
drive.mount('/content/gdrive')

warnings.filterwarnings('ignore')

data_df = pd.read_csv('DATASET.tsv', sep='\t', encoding='utf-8') # specify the stance dataset in the google drive

train_df, test_df = train_test_split(data_df, test_size=0.2) # create train/test splits

In [ ]:
# you can experiment with different inputs, 
# e.g., FIRST_ENTITY is good, SECOND_ENTITY is better (is good),
# take the whole question etc.

def transform_train(df):
    df["text_a"] = ['FIRST_ENTITY is better'] * len(df)
    # df["text_a"] = df["cleaned_question"] # to take the whole question before the [SEP] token
    df["text_b"] = df["masked_answer"] # the part after the [SEP] token
    df["labels"] = df["answer_stance"]
    return df

def get_classification_model(model):
    label_count = 4
    return ClassificationModel(model[0], model[1], num_labels=label_count, use_cuda=torch.cuda.is_available(), cuda_device=0)


# any available model at simpletransformers can be used for experiments

#full_model = get_classification_model(['longformer', 'allenai/longformer-base-4096'])
full_model = get_classification_model(['roberta', 'roberta-large'])
train_df = transform_train(train_df).sample(frac=1)
args = {"overwrite_output_dir": True, "num_train_epochs": 5, "fp16": False, "train_batch_size": 4, "gradient_accumulation_steps": 4, "evaluate_during_training": False, 
        "learning_rate": 2e-5, "early_stopping_consider_epochs": True, "reprocess_input_data": True,
        'save_eval_checkpoints': False, 'save_model_every_epoch' : False,	'save_optimizer_and_scheduler': False, 	'save_steps': -1}
full_model.train_model(train_df, args=args)

test_df = transform_train(test_df)
_, raw_outputs, _ = full_model.eval_model(test_df)
test_probs = softmax(raw_outputs, axis=1)
test_preds = np.argmax(raw_outputs, axis=1)

In [ ]:
# print evaluation results
print(classification_report(test_df['labels'].tolist(), test_preds))